In [266]:
import json

In [159]:
data_sets = [ 'baseline', 'iot', 'stocks', 'movielens' ]
databases = [ 'influxdb', 'opentsdb', 'kairosdb-cassandra', 'kairosdb-scylladb' ]

# Loading

Loading data into the database. Metrics saved: loading speed (points per second) and total size of the data inside the database.

In [183]:
loading_speeds = { # Mean point insertion rate (per second)
    "influxdb": {
        "baseline": 481818,
        "movielens":  4342,
        "stocks": 156498,
        "iot": 317999,
    },
    "opentsdb": {
        "baseline": 89360,
        "movielens": 21196,
        "stocks": 86578,
        "iot": 162473,
    },
    "kairosdb-cassandra": {
        "baseline": 54792,
        "movielens": 29913, # Did succeed, but could not verify count
        "stocks": 78198,
        "iot": 87413,
    },
    "kairosdb-scylladb": {
        "baseline":  59231,
        "movielens": 0, # Did not succeed - not all points were saved
        "stocks":  82535,
        "iot": 98736,
    }
}

In [179]:
bytes_per_dp = {
    "csv": {
        "baseline": 571824569 / 10000000,
        "movielens": 803233084 / 10000000,
        "stocks": 294354837 / 10000000,
        "iot": 344793310 / 10000000,
    },
    "influxdb": {
        "baseline": 81762020 / 990000,
        "movielens": 577506112 / 990000,
        "stocks": 169634300 / 990000,
        "iot": 113695704 / 990000,
    },
    "opentsdb": {
        "baseline": 138454917 / 1000000,
        "movielens": 71566746 / 1000000,
        "stocks": 46119566 / 1000000,
        "iot": 8912765 / 1000000,
    },
    "kairosdb-cassandra": {
        "baseline": 154298112 / 1000000,
        "movielens": 264238436 / 1000000,
        "stocks": 182716189 / 1000000,
        "iot": 77072011 / 1000000,
    },
    "kairosdb-scylladb": {
        "baseline": 1074734152 / 1000000,
        "movielens": 0, #1074734153 * (1000000/274149),
        "stocks": 1074734122 / 1000000,
        "iot": 1074734170 / 1000000,
    }
}

In [270]:
# Table for storage efficiency

print(' & '.join(['Data set', 'csv'] + databases) + ' \\\\')

for ds in data_sets:
    print(ds + ' & ' + ' & '.join(map(lambda db: str(round(bytes_per_dp[db][ds], 3)), ['csv'] + databases)) + ' \\\\')


Data set & csv & influxdb & opentsdb & kairosdb-cassandra & kairosdb-scylladb \\
baseline & 57.182 & 82.588 & 138.455 & 154.298 & 1074.734 \\
iot & 34.479 & 114.844 & 8.913 & 77.072 & 1074.734 \\
stocks & 29.435 & 171.348 & 46.12 & 182.716 & 1074.734 \\
movielens & 80.323 & 583.34 & 71.567 & 264.238 & 0 \\


In [276]:
# Table for relative storage efficiency

print(' & '.join(['Data set', 'csv'] + databases) + ' \\\\')

for ds in data_sets:
    print(ds + ' & ' + ' & '.join(map(lambda db: str(round(bytes_per_dp[db][ds] / bytes_per_dp['csv'][ds], 4)), ['csv'] + databases)) + ' \\\\')


Data set & csv & influxdb & opentsdb & kairosdb-cassandra & kairosdb-scylladb \\
baseline & 1.0 & 1.4443 & 2.4213 & 2.6983 & 18.7948 \\
iot & 1.0 & 3.3308 & 0.2585 & 2.2353 & 31.1704 \\
stocks & 1.0 & 5.8211 & 1.5668 & 6.2073 & 36.5115 \\
movielens & 1.0 & 7.2624 & 0.891 & 3.2897 & 0.0 \\


In [184]:
# Table for data loading speed

print(' & '.join(['Data set'] + databases) + ' \\\\')

for ds in data_sets:
    print(ds + ' & ' + ' & '.join(map(lambda db: str(loading_speeds[db][ds]), databases)) + ' \\\\')



Data set & influxdb & opentsdb & kairosdb-cassandra & kairosdb-scylladb \\
baseline & 481818 & 89360 & 54792 & 59231 \\
iot & 317999 & 162473 & 87413 & 98736 \\
stocks & 156498 & 86578 & 78198 & 82535 \\
movielens & 4342 & 21196 & 29913 & 0 \\


In [220]:
# Table for relative data loading speed

print(' & '.join(['Data set'] + databases) + ' \\\\')

for ds in data_sets:
    print(ds + ' & ' + ' & '.join(map(lambda db: str(loading_speeds[db][ds] / loading_speeds['influxdb'][ds]), databases)) + ' \\\\')



Data set & influxdb & opentsdb & kairosdb-cassandra & kairosdb-scylladb \\
baseline & 1.0 & 0.1854642209298947 & 0.11371928819595781 & 0.12293231054049454 \\
iot & 1.0 & 0.5109229903238689 & 0.27488451221544724 & 0.31049154242623406 \\
stocks & 1.0 & 0.5532211274265486 & 0.4996741172411149 & 0.5273869314623829 \\
movielens & 1.0 & 4.881621372639337 & 6.889221556886228 & 0.0 \\


# OLAP

In [253]:
results = {
    "baseline": {
        "config": {
            "vegeta": { "duration": 120,"rate": 10, "timeout": 30 }
        },
        "opentsdb": {
            "smoke_test": 347.367, # fetches per s, http_load -parallel 10 -timeout 30 -seconds 30
            "vegeta": json.loads('{"latencies":{"total":15070522122,"mean":12558768,"50th":8760884,"95th":12852900,"99th":37494724,"max":825906077},"bytes_in":{"total":151200,"mean":126},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-26T10:44:23.023366412-06:00","latest":"2019-05-26T10:46:22.923150005-06:00","end":"2019-05-26T10:46:22.931529278-06:00","duration":119899783593,"wait":8379273,"requests":1200,"rate":10.008358347613052,"success":1,"status_codes":{"200":1200},"errors":[]}'),
            # rate 50"vegeta": json.loads('{"latencies":{"total":46736614363,"mean":7789435,"50th":6889286,"95th":8644399,"99th":27587592,"max":281790347},"bytes_in":{"total":756000,"mean":126},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-25T10:13:22.360875412-06:00","latest":"2019-05-25T10:15:22.340766535-06:00","end":"2019-05-25T10:15:22.347386656-06:00","duration":119979891123,"wait":6620121,"requests":6000,"rate":50.008380103037176,"success":1,"status_codes":{"200":6000},"errors":[]}')
            "docker_stats": True,
        },
        "influxdb": {
            "smoke_test": 6400.36,
            "vegeta": json.loads('{"latencies":{"total":1519004957,"mean":1265837,"50th":1231827,"95th":1399121,"99th":1775473,"max":7419689},"bytes_in":{"total":222000,"mean":185},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-26T10:49:23.68481563-06:00","latest":"2019-05-26T10:51:23.584793307-06:00","end":"2019-05-26T10:51:23.585931121-06:00","duration":119899977677,"wait":1137814,"requests":1200,"rate":10.008342146924285,"success":1,"status_codes":{"200":1200},"errors":[]}'),
            # rate 50 "vegeta": json.loads('{"latencies":{"total":7494332523,"mean":1249055,"50th":1211344,"95th":1418718,"99th":1800635,"max":7868000},"bytes_in":{"total":1110000,"mean":185},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-25T10:25:20.525863847-06:00","latest":"2019-05-25T10:27:20.505831706-06:00","end":"2019-05-25T10:27:20.507041622-06:00","duration":119979967859,"wait":1209916,"requests":6000,"rate":50.00834811900581,"success":1,"status_codes":{"200":6000},"errors":[]}')
            "docker_stats": True,
        },
        "kairosdb-cassandra": {
            "smoke_test": 12.8,
            "vegeta": json.loads('{"latencies":{"total":1035171303072,"mean":862642752,"50th":77400115,"95th":124640305,"99th":30000239647,"max":30000337026},"bytes_in":{"total":236946,"mean":197.455},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-26T10:33:11.934750576-06:00","latest":"2019-05-26T10:35:11.834672078-06:00","end":"2019-05-26T10:35:12.03257532-06:00","duration":119899921502,"wait":197903242,"requests":1200,"rate":10.008346835990075,"success":0.9775,"status_codes":{"0":27,"200":1173},"errors":["Get http://10.2.0.42:8080/api/v1/datapoints/query?query=%7B%22start_absolute%22%3A1546300800000%2C%22end_absolute%22%3A1546300803000%2C%22metrics%22%3A%5B%7B%22name%22%3A%22benchmark%22%2C%22tags%22%3A%7B%22tag1%22%3A%22TAG_1_8%22%2C%22tag2%22%3A%22TAG_2_6%22%7D%7D%5D%7D: net/http: request canceled (Client.Timeout exceeded while awaiting headers)"]}')
        },
        "kairosdb-scylladb": {
            "smoke_test": 15.4667,
            "vegeta": json.loads('{"latencies":{"total":276149618488,"mean":230124682,"50th":75721768,"95th":82588156,"99th":214445526,"max":30000383277},"bytes_in":{"total":241592,"mean":201.32666666666665},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-27T14:38:55.434784317-06:00","latest":"2019-05-27T14:40:55.334782028-06:00","end":"2019-05-27T14:40:55.63935822-06:00","duration":119899997711,"wait":304576192,"requests":1200,"rate":10.00834047463796,"success":0.9966666666666667,"status_codes":{"0":4,"200":1196},"errors":["Get http://10.2.0.42:8080/api/v1/datapoints/query?query=%7B%22start_absolute%22%3A1546300800000%2C%22end_absolute%22%3A1546300803000%2C%22metrics%22%3A%5B%7B%22name%22%3A%22benchmark%22%2C%22tags%22%3A%7B%22tag1%22%3A%22TAG_1_8%22%2C%22tag2%22%3A%22TAG_2_6%22%7D%7D%5D%7D: net/http: request canceled (Client.Timeout exceeded while awaiting headers)"]}')
            #"vegeta": json.loads('{"latencies":{"total":164198015566,"mean":136831679,"50th":64214704,"95th":70532287,"99th":82608986,"max":30000160027},"bytes_in":{"total":103028,"mean":85.85666666666667},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-26T10:39:17.472709847-06:00","latest":"2019-05-26T10:41:17.372550086-06:00","end":"2019-05-26T10:41:17.43940145-06:00","duration":119899840239,"wait":66851364,"requests":1200,"rate":10.008353619220872,"success":0.9983333333333333,"status_codes":{"0":2,"200":1198},"errors":["Get http://10.2.0.42:8080/api/v1/datapoints/query?query=%7B%22start_absolute%22%3A1546300800000%2C%22end_absolute%22%3A1546300803000%2C%22metrics%22%3A%5B%7B%22name%22%3A%22benchmark%22%2C%22tags%22%3A%7B%22tag1%22%3A%22TAG_1_8%22%2C%22tag2%22%3A%22TAG_2_6%22%7D%7D%5D%7D: net/http: request canceled (Client.Timeout exceeded while awaiting headers)"]}')
        },
    },
    "movielens": {
        "config": {
            "vegeta": { "duration": 120,"rate": 2, "timeout": 30 }
        },
        "opentsdb": {
            "smoke_test": 2.13333,
            "vegeta": json.loads('{"latencies":{"total":615124808278,"mean":2563020034,"50th":2557478376,"95th":2786607644,"99th":2874002844,"max":3115421721},"bytes_in":{"total":93756,"mean":390.65},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-26T11:15:35.720640068-06:00","latest":"2019-05-26T11:17:35.220619818-06:00","end":"2019-05-26T11:17:37.394694003-06:00","duration":119499979750,"wait":2174074185,"requests":240,"rate":2.008368541167054,"success":1,"status_codes":{"200":240},"errors":[]}'),
            "docker_stats": True,
        },
        "influxdb": {
            "smoke_test": 78.3333,
            "vegeta": json.loads('{"latencies":{"total":25058484333,"mean":104410351,"50th":35781987,"95th":462786983,"99th":571478623,"max":748210568},"bytes_in":{"total":300084,"mean":1250.35},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-26T11:49:43.743733081-06:00","latest":"2019-05-26T11:51:43.243720962-06:00","end":"2019-05-26T11:51:43.285025786-06:00","duration":119499987881,"wait":41304824,"requests":240,"rate":2.008368404513947,"success":1,"status_codes":{"200":240},"errors":[]}'),
            "docker_stats": True,
        },
        "kairosdb-cassandra": {
            "smoke_test": 0, # Could not complete! org.kairosdb.core.exception.DatastoreException: org.kairosdb.util.MemoryMonitorException: Insufficient memory to perform request
            #"vegeta": json.loads('')
        },
        "kairosdb-scylladb": {
            "smoke_test": 0, # Could not complete! Could not load data set
            #"vegeta": json.loads('')
        },
    },
    "stocks": {
        "config": {
            "vegeta": { "duration": 120,"rate": 20, "timeout": 0 }
        },
        "opentsdb": {
            "smoke_test": 15.5664,
            "vegeta": json.loads('{"latencies":{"total":6435457340252,"mean":2681440558,"50th":2390509199,"95th":3991057059,"99th":4285139552,"max":4864904304},"bytes_in":{"total":69250440,"mean":28854.35},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-25T12:46:09.002424471-06:00","latest":"2019-05-25T12:48:08.952376236-06:00","end":"2019-05-25T12:48:10.30802461-06:00","duration":119949951765,"wait":1355648374,"requests":2400,"rate":20.008344852876316,"success":1,"status_codes":{"200":2400},"errors":[]}'),
            "docker_stats": True,
        },
        "influxdb": {
            "smoke_test": 235.733,
            "vegeta": json.loads('{"latencies":{"total":138911618384,"mean":57879840,"50th":55517268,"95th":87277173,"99th":101453192,"max":121655629},"bytes_in":{"total":79646640,"mean":33186.1},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-25T12:41:13.5943453-06:00","latest":"2019-05-25T12:43:13.544312501-06:00","end":"2019-05-25T12:43:13.600544309-06:00","duration":119949967201,"wait":56231808,"requests":2400,"rate":20.008342278062678,"success":1,"status_codes":{"200":2400},"errors":[]}'),
            "docker_stats": True
        },
        "kairosdb-cassandra": {
            "smoke_test": 26.8332,
            "vegeta": json.loads('{"latencies":{"total":256440660663,"mean":106850275,"50th":102376135,"95th":133174898,"99th":148262821,"max":234225101},"bytes_in":{"total":55482600,"mean":23117.75},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-25T13:07:43.66250197-06:00","latest":"2019-05-25T13:09:43.612474067-06:00","end":"2019-05-25T13:09:43.697448085-06:00","duration":119949972097,"wait":84974018,"requests":2400,"rate":20.00834146138184,"success":1,"status_codes":{"200":2400},"errors":[]}')
        },
        "kairosdb-scylladb": {
            "smoke_test": 28.5667,
            "vegeta": json.loads('{"latencies":{"total":293399166031,"mean":122249652,"50th":121426422,"95th":136843726,"99th":148021624,"max":170815253},"bytes_in":{"total":55482600,"mean":23117.75},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-27T15:28:30.237693354-06:00","latest":"2019-05-27T15:30:30.187762774-06:00","end":"2019-05-27T15:30:30.284325453-06:00","duration":119950069420,"wait":96562679,"requests":2400,"rate":20.008325227361922,"success":1,"status_codes":{"200":2400},"errors":[]}'),
            #"vegeta": json.loads('{"latencies":{"total":245094234589,"mean":102122597,"50th":98576673,"95th":127224758,"99th":146029928,"max":286352096},"bytes_in":{"total":51888600,"mean":21620.25},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-25T13:18:56.34748875-06:00","latest":"2019-05-25T13:20:56.297586349-06:00","end":"2019-05-25T13:20:56.386031861-06:00","duration":119950097599,"wait":88445512,"requests":2400,"rate":20.00832052695227,"success":1,"status_codes":{"200":2400},"errors":[]}')
        },
    },
    "iot": {
        "config": {
            "vegeta": { "duration": 120,"rate": 30, "timeout": 30 }
        },
        "opentsdb": {
            "smoke_test": 117.3, # reqs/s
            #"vegeta": json.loads('{"latencies":{"total":299779722389,"mean":24981643,"50th":10477034,"95th":94561740,"99th":109841776,"max":173976168},"bytes_in":{"total":4205400,"mean":350.45},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-25T14:02:49.983261973-06:00","latest":"2019-05-25T14:04:49.973013938-06:00","end":"2019-05-25T14:04:50.044311643-06:00","duration":119989751965,"wait":71297705,"requests":12000,"rate":100.00854075854994,"success":1,"status_codes":{"200":12000},"errors":[]}')
            "vegeta": json.loads('{"latencies":{"total":65856330367,"mean":18293425,"50th":7688133,"95th":44926242,"99th":103119224,"max":1000546784},"bytes_in":{"total":1261620,"mean":350.45},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-25T14:56:15.067189689-06:00","latest":"2019-05-25T14:58:15.03369499-06:00","end":"2019-05-25T14:58:15.072744873-06:00","duration":119966505301,"wait":39049883,"requests":3600,"rate":30.00837601268353,"success":1,"status_codes":{"200":3600},"errors":[]}'),
            "docker_stats": True,
        },
        "influxdb": {
            "smoke_test": 997.567,
            #"vegeta": json.loads('{"latencies":{"total":88749824295,"mean":7395818,"50th":4524806,"95th":21920820,"99th":26154907,"max":38884255},"bytes_in":{"total":6088200,"mean":507.35},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-25T14:06:54.269413001-06:00","latest":"2019-05-25T14:08:54.259327209-06:00","end":"2019-05-25T14:08:54.278129048-06:00","duration":119989914208,"wait":18801839,"requests":12000,"rate":100.00840553313715,"success":1,"status_codes":{"200":12000},"errors":[]}')
            "vegeta": json.loads('{"latencies":{"total":27489685505,"mean":7636023,"50th":5310755,"95th":22049411,"99th":23346706,"max":30482868},"bytes_in":{"total":1826460,"mean":507.35},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-25T14:52:10.068214027-06:00","latest":"2019-05-25T14:54:10.034853599-06:00","end":"2019-05-25T14:54:10.056127436-06:00","duration":119966639572,"wait":21273837,"requests":3600,"rate":30.008342426224242,"success":1,"status_codes":{"200":3600},"errors":[]}'),
            "docker_stats": True,
        },
        "kairosdb-cassandra": {
            "smoke_test": 29.4333,
            "vegeta": json.loads('{"latencies":{"total":561275367174,"mean":155909824,"50th":120016576,"95th":333995350,"99th":367150766,"max":463262969},"bytes_in":{"total":1653840,"mean":459.4},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-25T14:27:44.763563967-06:00","latest":"2019-05-25T14:29:44.73018003-06:00","end":"2019-05-25T14:29:45.019319655-06:00","duration":119966616063,"wait":289139625,"requests":3600,"rate":30.00834830674455,"success":1,"status_codes":{"200":3600},"errors":[]}'),
            #"docker_stats": ,
        },
        "kairosdb-scylladb": {
            "smoke_test": 40.1667,
            "vegeta": json.loads('{"latencies":{"total":268164593577,"mean":74490164,"50th":53901595,"95th":193673539,"99th":210816087,"max":343825017},"bytes_in":{"total":1653840,"mean":459.4},"bytes_out":{"total":0,"mean":0},"earliest":"2019-05-25T14:40:34.345559774-06:00","latest":"2019-05-25T14:42:34.312372285-06:00","end":"2019-05-25T14:42:34.482179383-06:00","duration":119966812511,"wait":169807098,"requests":3600,"rate":30.00829916748775,"success":1,"status_codes":{"200":3600},"errors":[]}'),
            #"docker_stats": ,
        },
    }
}

In [254]:
# Table for max reqs/s

print(' & '.join(['Data set'] + databases) + ' \\\\')

for ds in data_sets:
    print(ds + ' & ' + ' & '.join(map(lambda db: str(results[ds][db]['smoke_test']), databases)) + ' \\\\')



Data set & influxdb & opentsdb & kairosdb-cassandra & kairosdb-scylladb \\
baseline & 6400.36 & 347.367 & 12.8 & 15.4667 \\
iot & 997.567 & 117.3 & 29.4333 & 40.1667 \\
stocks & 235.733 & 15.5664 & 26.8332 & 28.5667 \\
movielens & 78.3333 & 2.13333 & 0 & 0 \\


In [256]:
# Table for relative max reqs/s

print(' & '.join(['Data set'] + databases) + ' \\\\')

for ds in data_sets:
    print(ds + ' & ' + ' & '.join(map(lambda db: str(results[ds][db]['smoke_test'] / results[ds]['influxdb']['smoke_test']), databases)) + ' \\\\')



Data set & influxdb & opentsdb & kairosdb-cassandra & kairosdb-scylladb \\
baseline & 1.0 & 0.05427304089144986 & 0.0019998875063277693 & 0.002416535944853102 \\
iot & 1.0 & 0.11758608694954825 & 0.029505085873931275 & 0.04026466392733521 \\
stocks & 1.0 & 0.06603403002549495 & 0.11382878086648879 & 0.12118243945480692 \\
movielens & 1.0 & 0.027234011588941103 & 0.0 & 0.0 \\


In [247]:
# Table for mean bytes per response

print(' & '.join(['Data set'] + databases) + ' \\\\')

for ds in data_sets:
    def get_bytes_in(db_details):
        if 'vegeta' in db_details:
            return db_details['vegeta']['bytes_in']['total'] / db_details['vegeta']['status_codes']['200']
        else:
            return 0
    
    print(ds + ' & ' + ' & '.join(map(lambda db: str(get_bytes_in(results[ds][db])), databases)) + ' \\\\')


Data set & influxdb & opentsdb & kairosdb-cassandra & kairosdb-scylladb \\
baseline & 185.0 & 126.0 & 202.0 & 202.0 \\
iot & 507.35 & 350.45 & 459.4 & 459.4 \\
stocks & 33186.1 & 28854.35 & 23117.75 & 23117.75 \\
movielens & 1250.35 & 390.65 & 0 & 0 \\


In [212]:
def ns_to_ms(nanoseconds):
    return nanoseconds / (10**6)

In [278]:
# Table for mean request latency

print(' & '.join(['Data set'] + databases) + ' \\\\')

for ds in data_sets:
    def get_mean_latency(db_details):
        if 'vegeta' in db_details:
            return ns_to_ms(db_details['vegeta']['latencies']['mean'])
        else:
            return 0
    
    print(ds + ' & ' + ' & '.join(map(lambda db: str(round(get_mean_latency(results[ds][db]), 3)), databases)) + ' \\\\')


Data set & influxdb & opentsdb & kairosdb-cassandra & kairosdb-scylladb \\
baseline & 1.266 & 12.559 & 862.643 & 230.125 \\
iot & 7.636 & 18.293 & 155.91 & 74.49 \\
stocks & 57.88 & 2681.441 & 106.85 & 122.25 \\
movielens & 104.41 & 2563.02 & 0 & 0 \\


In [255]:
# Table for number of time-outs

print(' & '.join(['Data set'] + databases) + ' \\\\')

for ds in data_sets:
    def get_mean_latency(db_details):
        if 'vegeta' in db_details and '0' in db_details['vegeta']['status_codes']:
            return db_details['vegeta']['status_codes']['0']
        else:
            return 0
    
    print(ds + ' & ' + ' & '.join(map(lambda db: str(get_mean_latency(results[ds][db])), databases)) + ' \\\\')


Data set & influxdb & opentsdb & kairosdb-cassandra & kairosdb-scylladb \\
baseline & 0 & 0 & 27 & 4 \\
iot & 0 & 0 & 0 & 0 \\
stocks & 0 & 0 & 0 & 0 \\
movielens & 0 & 0 & 0 & 0 \\


In [216]:
# Table for 95th percentile request latency

print(' & '.join(['Data set'] + databases) + ' \\\\')

for ds in data_sets:
    def get_mean_latency(db_details):
        if 'vegeta' in db_details:
            return ns_to_ms(db_details['vegeta']['latencies']['95th'])
        else:
            return 0
    
    print(ds + ' & ' + ' & '.join(map(lambda db: str(get_mean_latency(results[ds][db])), databases)) + ' \\\\')


Data set & influxdb & opentsdb & kairosdb-cassandra & kairosdb-scylladb \\
baseline & 1.399121 & 12.8529 & 124.640305 & 70.532287 \\
iot & 22.049411 & 44.926242 & 333.99535 & 193.673539 \\
stocks & 87.277173 & 3991.057059 & 133.174898 & 127.224758 \\
movielens & 462.786983 & 2786.607644 & 0 & 0 \\
